In [4]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
#URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [10]:
import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [5]:
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
model = MultVAERecommender

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
earlystopping_kwargs =  {"validation_every_n": 10,
                              "stop_on_validation": True,
                              "evaluator_object": evaluator_validation,
                              "lower_validations_allowed": 5,
                              "validation_metric": "MAP",
                              }

EvaluatorHoldout: Ignoring 2147 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [5]:
import optuna as op
def objective(trial):
    n_users, n_items = URM_train_validation.shape
    epochs = trial.suggest_int("epochs", 250, 250)
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-2, log=True)
    l2_reg = trial.suggest_float("l2_reg", 1e-6, 1e-2, log=True)
    dropout = trial.suggest_float("dropout", 0., 0.8)
    total_anneal_steps = trial.suggest_int("total_anneal_steps", 100000, 600000)
    anneal_cap = trial.suggest_float("anneal_cap", 0., 0.6)
    batch_size = trial.suggest_categorical("batch_size", [128, 256, 512, 1024])
    
    recommender = model(URM_train_validation, force_gpu=True)
    recommender.fit(**trial.params, **earlystopping_kwargs)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [6]:
best_params = {
    "epochs": 500,
    "learning_rate": 0.0001,
    "l2_reg": 0.0001,
    "dropout": 0.2,
    "total_anneal_steps": 200000,
    "anneal_cap": 0.2,
    "batch_size": 512,
    "encoding_size": 128,
    "next_layer_size_multiplier": 2,
    "max_n_hidden_layers": 2,
    "max_parameters": 7*1e9*8/32
}

study_name = "multvae2-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
#op.delete_study(study_name, storage_name)
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
#study.enqueue_trial(best_params)
study.optimize(objective, n_trials=200)

[I 2023-12-18 23:29:30,426] Using an existing study with name 'multvae2-study' instead of creating a new one.


MultVAERecommender: URM Detected 205 ( 1.6%) users with no interactions.
MultVAERecommender: URM Detected 126 ( 0.6%) items with no interactions.
MultVAERecommender: Using default Temp folder './result_experiments/__Temp_MultVAERecommender_2512/'


2023-12-18 23:29:31.089905: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5730 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6
2023-12-18 23:29:31.091238: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-12-18 23:29:31.436549: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


MultVAERecommender: Saving model in file './result_experiments/__Temp_MultVAERecommender_2512/_best_model'
MultVAERecommender: Saving complete


[W 2023-12-18 23:29:33,975] Trial 262 failed with parameters: {'epochs': 250, 'learning_rate': 0.001109446389530806, 'l2_reg': 2.3892681267208043e-05, 'dropout': 0.11105171795593702, 'total_anneal_steps': 125513, 'anneal_cap': 0.25789568369869914, 'batch_size': 128} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/alessandro/RecSys-2023-polimi/venv/lib/python3.11/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_2512/2185333188.py", line 13, in objective
    recommender.fit(**trial.params, **earlystopping_kwargs)
  File "/home/alessandro/RecSys-2023-polimi/Recommenders/Neural/MultVAERecommender.py", line 311, in fit
    self._train_with_early_stopping(epochs,
  File "/home/alessandro/RecSys-2023-polimi/Recommenders/Incremental_Training_Early_Stopping.py", line 200, in _train_with_early_stopping
    self._run_epoch(epochs_current

MultVAERecommender: Cleaning temporary files from './result_experiments/__Temp_MultVAERecommender_2512/'


KeyboardInterrupt: 

In [7]:
study.best_params

{'epochs': 250,
 'learning_rate': 0.002119915084944386,
 'l2_reg': 0.00014575953906446464,
 'dropout': 0.11923360650750325,
 'total_anneal_steps': 155145,
 'anneal_cap': 0.29170819619489025,
 'batch_size': 128}

In [10]:
final = model(URM_all)
final.fit(epochs=150, learning_rate=0.002119915084944386, l2_reg=0.00014575953906446464, dropout=0.11923360650750325, total_anneal_steps=155145, anneal_cap=0.29170819619489025,batch_size=128)#, **earlystopping_kwargs)

MultVAERecommender: Using default Temp folder './result_experiments/__Temp_MultVAERecommender_2512/'


2023-12-18 23:40:07.487763: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5730 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


MultVAERecommender: Saving model in file './result_experiments/__Temp_MultVAERecommender_2512/_best_model'
MultVAERecommender: Saving complete
MultVAERecommender: Epoch 1 of 150. Elapsed time 1.95 sec
MultVAERecommender: Epoch 2 of 150. Elapsed time 2.81 sec
MultVAERecommender: Epoch 3 of 150. Elapsed time 3.66 sec
MultVAERecommender: Epoch 4 of 150. Elapsed time 4.52 sec
MultVAERecommender: Epoch 5 of 150. Elapsed time 5.38 sec
MultVAERecommender: Epoch 6 of 150. Elapsed time 6.23 sec
MultVAERecommender: Epoch 7 of 150. Elapsed time 7.08 sec
MultVAERecommender: Epoch 8 of 150. Elapsed time 7.94 sec
MultVAERecommender: Epoch 9 of 150. Elapsed time 8.79 sec
MultVAERecommender: Epoch 10 of 150. Elapsed time 9.64 sec
MultVAERecommender: Epoch 11 of 150. Elapsed time 10.49 sec
MultVAERecommender: Epoch 12 of 150. Elapsed time 11.35 sec
MultVAERecommender: Epoch 13 of 150. Elapsed time 12.20 sec
MultVAERecommender: Epoch 14 of 150. Elapsed time 13.05 sec
MultVAERecommender: Epoch 15 of 150.

2023-12-18 23:42:18.111183: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5730 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


INFO:tensorflow:Restoring parameters from ./result_experiments/__Temp_MultVAERecommender_2512/_best_model/.session/session
MultVAERecommender: Loading complete
MultVAERecommender: Cleaning temporary files from './result_experiments/__Temp_MultVAERecommender_2512/'


In [13]:
final.save_model(folder_path="slim_models/", file_name="MultVAE_all")

MultVAERecommender: Saving model in file 'slim_models/MultVAE_all'
MultVAERecommender: Saving complete


In [9]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 2181 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10457 (100.0%) in 6.15 sec. Users per second: 1701


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10       0.06368                 0.118407  0.103261  0.029278    0.053807   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.195824  0.103339  0.078778  0.424022      0.237966  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.827425          0.350847    0.827425       0.022418   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10            8.020277                   0.983122             0.064258   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   

In [12]:
from Recommenders.Neural.MultVAERecommender import MultVAERecommender_OptimizerMask
model = MultVAERecommender_OptimizerMask
import optuna as op
def objective(trial):
    n_users, n_items = URM_train_validation.shape
    epochs = trial.suggest_int("epochs", 350, 350)
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-2, log=True)
    l2_reg = trial.suggest_float("l2_reg", 1e-6, 1e-2, log=True)
    dropout = trial.suggest_float("dropout", 0., 0.8)
    total_anneal_steps = trial.suggest_int("total_anneal_steps", 100000, 600000)
    anneal_cap = trial.suggest_float("anneal_cap", 0., 0.6)
    batch_size = trial.suggest_categorical("batch_size", [128, 256, 512, 1024])
    
    encoding_size = trial.suggest_int("encoding_size", 1, min(512, n_items-1))
    next_layer_size_multiplier = trial.suggest_int("next_layer_size_multiplier", 2, 10)
    max_n_hidden_layers = trial.suggest_int("max_n_hidden_layers", 1, 4)
    
    max_parameters = trial.suggest_categorical("max_parameters", [12*1e9*8/32])
    
    recommender = model(URM_train_validation, force_gpu=True)
    recommender.fit(**trial.params, **earlystopping_kwargs)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [ ]:
study_name = "multvae-opt-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
#op.delete_study(study_name, storage_name)
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
study.optimize(objective, n_trials=200)

[I 2023-12-25 11:50:39,971] Using an existing study with name 'multvae-opt-study' instead of creating a new one.
2023-12-25 11:50:40.117008: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5449 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


MultVAERecommender: URM Detected 208 ( 1.6%) users with no interactions.
MultVAERecommender: URM Detected 100 ( 0.5%) items with no interactions.
MultVAERecommender: Architecture: [427, 22222]
MultVAERecommender: Using default Temp folder './result_experiments/__Temp_MultVAERecommender_222264/'
MultVAERecommender: Saving model in file './result_experiments/__Temp_MultVAERecommender_222264/_best_model'
MultVAERecommender: Saving complete
MultVAERecommender: Epoch 1 of 350. Elapsed time 17.61 sec
MultVAERecommender: Epoch 2 of 350. Elapsed time 36.05 sec
MultVAERecommender: Epoch 3 of 350. Elapsed time 58.27 sec
MultVAERecommender: Epoch 4 of 350. Elapsed time 1.28 min
MultVAERecommender: Epoch 5 of 350. Elapsed time 1.61 min
MultVAERecommender: Epoch 6 of 350. Elapsed time 1.90 min
MultVAERecommender: Epoch 7 of 350. Elapsed time 2.24 min
MultVAERecommender: Epoch 8 of 350. Elapsed time 2.60 min
MultVAERecommender: Epoch 9 of 350. Elapsed time 2.91 min
MultVAERecommender: Validation beg

2023-12-25 13:58:02.642022: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5449 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


INFO:tensorflow:Restoring parameters from ./result_experiments/__Temp_MultVAERecommender_222264/_best_model/.session/session
MultVAERecommender: Loading complete
MultVAERecommender: Cleaning temporary files from './result_experiments/__Temp_MultVAERecommender_222264/'
EvaluatorHoldout: Processed 10467 (100.0%) in 59.23 sec. Users per second: 177


[I 2023-12-25 13:59:02,207] Trial 76 finished with value: 0.03496835860550423 and parameters: {'epochs': 350, 'learning_rate': 0.0002522900964332827, 'l2_reg': 0.00022747374615059398, 'dropout': 0.5609240460203853, 'total_anneal_steps': 301186, 'anneal_cap': 0.2822984588808496, 'batch_size': 1024, 'encoding_size': 427, 'next_layer_size_multiplier': 10, 'max_n_hidden_layers': 4, 'max_parameters': 3000000000.0}. Best is trial 49 with value: 0.03611212744960291.


...

INFO:tensorflow:Restoring parameters from ./result_experiments/__Temp_MultVAERecommender_222264/_best_model/.session/session
MultVAERecommender: Loading complete
MultVAERecommender: Cleaning temporary files from './result_experiments/__Temp_MultVAERecommender_222264/'
EvaluatorHoldout: Processed 10467 (100.0%) in 1.65 min. Users per second: 106


[I 2023-12-25 23:28:06,553] Trial 83 finished with value: 0.03353593152174448 and parameters: {'epochs': 350, 'learning_rate': 0.000605003985326604, 'l2_reg': 0.0006479653689219195, 'dropout': 0.5365401133452798, 'total_anneal_steps': 533873, 'anneal_cap': 0.3674253573041222, 'batch_size': 128, 'encoding_size': 436, 'next_layer_size_multiplier': 9, 'max_n_hidden_layers': 3, 'max_parameters': 3000000000.0}. Best is trial 49 with value: 0.03611212744960291.


MultVAERecommender: URM Detected 208 ( 1.6%) users with no interactions.
MultVAERecommender: URM Detected 100 ( 0.5%) items with no interactions.
MultVAERecommender: Architecture: [338, 22222]
MultVAERecommender: Using default Temp folder './result_experiments/__Temp_MultVAERecommender_222264/'


2023-12-25 23:28:06.645268: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5449 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


MultVAERecommender: Saving model in file './result_experiments/__Temp_MultVAERecommender_222264/_best_model'
MultVAERecommender: Saving complete
MultVAERecommender: Epoch 1 of 350. Elapsed time 20.41 sec
MultVAERecommender: Epoch 2 of 350. Elapsed time 36.08 sec
MultVAERecommender: Epoch 3 of 350. Elapsed time 1.05 min
MultVAERecommender: Epoch 4 of 350. Elapsed time 1.31 min
MultVAERecommender: Epoch 5 of 350. Elapsed time 1.67 min
MultVAERecommender: Epoch 6 of 350. Elapsed time 2.10 min
MultVAERecommender: Epoch 7 of 350. Elapsed time 2.62 min
MultVAERecommender: Epoch 8 of 350. Elapsed time 3.11 min
MultVAERecommender: Epoch 9 of 350. Elapsed time 3.60 min
MultVAERecommender: Validation begins...
EvaluatorHoldout: Processed 10467 (100.0%) in 1.72 min. Users per second: 102
MultVAERecommender: epoch
10    0.011435
Name: MAP, dtype: object
MultVAERecommender: New best model found! Updating.
MultVAERecommender: Saving model in file './result_experiments/__Temp_MultVAERecommender_22226

In [3]:
from numba import cuda
device = cuda.select_device(0)
device.reset()

In [7]:
final = MultVAERecommender_OptimizerMask(URM_all)
final.fit(epochs=210, learning_rate=0.0002229220517428562, l2_reg=1.7199457884691567e-05, dropout=0.5397666797136971, total_anneal_steps=134713, anneal_cap=0.20025495518554443,batch_size=512,encoding_size=460,next_layer_size_multiplier=10,max_n_hidden_layers=4, max_parameters=3000000000.0)#, **earlystopping_kwargs)

MultVAERecommender: Architecture: [460, 22222]
MultVAERecommender: Using default Temp folder './result_experiments/__Temp_MultVAERecommender_2893/'


2023-12-26 00:34:01.952914: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-26 00:34:01.953163: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-26 00:34:01.953297: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

MultVAERecommender: Saving model in file './result_experiments/__Temp_MultVAERecommender_2893/_best_model'
MultVAERecommender: Saving complete
MultVAERecommender: Epoch 1 of 210. Elapsed time 1.91 sec
MultVAERecommender: Epoch 2 of 210. Elapsed time 2.78 sec
MultVAERecommender: Epoch 3 of 210. Elapsed time 3.65 sec
MultVAERecommender: Epoch 4 of 210. Elapsed time 4.51 sec
MultVAERecommender: Epoch 5 of 210. Elapsed time 5.39 sec
MultVAERecommender: Epoch 6 of 210. Elapsed time 6.27 sec
MultVAERecommender: Epoch 7 of 210. Elapsed time 7.14 sec
MultVAERecommender: Epoch 8 of 210. Elapsed time 8.01 sec
MultVAERecommender: Epoch 9 of 210. Elapsed time 8.88 sec
MultVAERecommender: Epoch 10 of 210. Elapsed time 9.75 sec
MultVAERecommender: Epoch 11 of 210. Elapsed time 10.62 sec
MultVAERecommender: Epoch 12 of 210. Elapsed time 11.49 sec
MultVAERecommender: Epoch 13 of 210. Elapsed time 12.36 sec
MultVAERecommender: Epoch 14 of 210. Elapsed time 13.23 sec
MultVAERecommender: Epoch 15 of 210.

2023-12-26 00:37:05.684711: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-26 00:37:05.684982: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-26 00:37:05.685157: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

INFO:tensorflow:Restoring parameters from ./result_experiments/__Temp_MultVAERecommender_2893/_best_model/.session/session
MultVAERecommender: Loading complete
MultVAERecommender: Cleaning temporary files from './result_experiments/__Temp_MultVAERecommender_2893/'


In [8]:
final.save_model(folder_path="slim_models/", file_name="MultVAE_all_opt")

MultVAERecommender: Saving model in file 'slim_models/MultVAE_all_opt'
MultVAERecommender: Saving complete


In [9]:
from Recommenders.Neural.MultVAERecommender import MultVAERecommender_OptimizerMask
final = MultVAERecommender_OptimizerMask(URM_train_validation)
final.fit(epochs=500, learning_rate=0.0002229220517428562, l2_reg=1.7199457884691567e-05, dropout=0.5397666797136971, total_anneal_steps=134713, anneal_cap=0.20025495518554443,batch_size=512,encoding_size=460,next_layer_size_multiplier=10,max_n_hidden_layers=4, max_parameters=3000000000.0)#, **earlystopping_kwargs)

MultVAERecommender: URM Detected 206 ( 1.6%) users with no interactions.
MultVAERecommender: URM Detected 99 ( 0.4%) items with no interactions.
MultVAERecommender: Architecture: [460, 22222]
MultVAERecommender: Using default Temp folder './result_experiments/__Temp_MultVAERecommender_2893/'


2023-12-26 00:44:06.987434: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-26 00:44:06.987665: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-26 00:44:06.987821: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

MultVAERecommender: Saving model in file './result_experiments/__Temp_MultVAERecommender_2893/_best_model'
MultVAERecommender: Saving complete
MultVAERecommender: Epoch 1 of 500. Elapsed time 2.07 sec
MultVAERecommender: Epoch 2 of 500. Elapsed time 2.91 sec
MultVAERecommender: Epoch 3 of 500. Elapsed time 3.75 sec
MultVAERecommender: Epoch 4 of 500. Elapsed time 4.58 sec
MultVAERecommender: Epoch 5 of 500. Elapsed time 5.41 sec
MultVAERecommender: Epoch 6 of 500. Elapsed time 6.24 sec
MultVAERecommender: Epoch 7 of 500. Elapsed time 7.08 sec
MultVAERecommender: Epoch 8 of 500. Elapsed time 7.91 sec
MultVAERecommender: Epoch 9 of 500. Elapsed time 8.74 sec
MultVAERecommender: Epoch 10 of 500. Elapsed time 9.58 sec
MultVAERecommender: Epoch 11 of 500. Elapsed time 10.41 sec
MultVAERecommender: Epoch 12 of 500. Elapsed time 11.25 sec
MultVAERecommender: Epoch 13 of 500. Elapsed time 12.09 sec
MultVAERecommender: Epoch 14 of 500. Elapsed time 12.92 sec
MultVAERecommender: Epoch 15 of 500.

2023-12-26 00:51:07.461309: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-26 00:51:07.461508: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-26 00:51:07.461633: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

INFO:tensorflow:Restoring parameters from ./result_experiments/__Temp_MultVAERecommender_2893/_best_model/.session/session
MultVAERecommender: Loading complete
MultVAERecommender: Cleaning temporary files from './result_experiments/__Temp_MultVAERecommender_2893/'
